In [85]:
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from functions import merge_data
from functions import load_medicare_data
from functions import load_respiratory_disease_data
from os.path import join as oj
import math
import re
import joblib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Adding FIPS to hospital data

In [82]:
#icu['cnty_name'] = icu.apply(lambda x: re.sub('[^a-zA-Z]+', '', x['cnty_name']).lower(), axis=1)
hospitals = pd.read_csv("data/hospital_level_info/02_hospital_info.csv")
county_to_fips = dict(zip(zip(hospitals['COUNTY'], hospitals['STATE']), hospitals['COUNTYFIPS']))
hospital_level_info_merged = pd.read_csv("data/hospital_level_info/03_hospital_level_info_merged.csv")
def map_county_to_fips(name, st):
    if type(name) is str:
        #name = re.sub('[^a-zA-Z]+', '', county_name).lower()
        if (name, st) in county_to_fips and county_to_fips[(name, st)] != "NOT AVAILABLE":
            return int(county_to_fips[(name, st)])
    return np.nan
    
hospital_level_info_merged['countyFIPS'] = hospital_level_info_merged.apply(lambda x: map_county_to_fips(x['County Name_y'], x['State_x']), 
                                                                            axis=1).astype('Int64')

In [83]:
len(set(hospital_level_info_merged['countyFIPS'].values))

2402

In [84]:
hospital_level_info_merged['countyFIPS']

0       1069
1       1095
2       1077
3       1039
4       1041
        ... 
7138    <NA>
7139    <NA>
7140    <NA>
7141    <NA>
7142    <NA>
Name: countyFIPS, Length: 7143, dtype: Int64